<a href="https://colab.research.google.com/github/sunshineharry/4dof_Gripper_controler/blob/master/Method_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 分割-识别法

采用如下方法，完成目标识别

- 先利用深度图像，提取含有物体的区域

- 将含有物体的区域进行目标分类 **仅仅做分类而不是识别**

- 将分类完成的局部图像和深度掩码融合，得到类似maskrcnn的效果

版本信息

- python == 3.6

- tensorflow == 1.15.2

- keras == 2.3.1

- opencv == 3.4.2.16 or 4.1.2.30

- h5py == 2.10.0

In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:
def show_image(window_name, image, time=0):
    """
    绘图函数，按Esc健关闭绘图窗口
    """
    # 解决uint16的问题
    if image.dtype == np.uint16:
        image = cv2.convertScaleAbs(image, alpha=(255.0/65535.0))    
    try:
        # 如果是彩色图像，转为 RGB 格式
        if image.shape[2] == 3:
            image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
    except:
        image = cv2.cvtColor(image,cv2.COLOR_GRAY2RGB)
    plt.imshow(image)
    plt.show()

## 深度图像处理

### 读取和裁剪图像

In [ ]:
#@title
# 读取背景深度图像

backgroud_depth = cv2.imread('dateset/NO_object/depth.png',cv2.IMREAD_UNCHANGED)
backgroud_depth_colormap = cv2.applyColorMap(cv2.convertScaleAbs(backgroud_depth, alpha=0.2), cv2.COLORMAP_JET)
show_image('',backgroud_depth_colormap)

# 裁剪ROI，即传送带部分
backgroud_depth = backgroud_depth[115:345,0:500]
backgroud_depth_colormap = cv2.applyColorMap(cv2.convertScaleAbs(backgroud_depth, alpha=0.2), cv2.COLORMAP_JET)
show_image('',backgroud_depth_colormap)

In [ ]:
# 读取传送带上有物体的深度图像
object_depth = cv2.imread('dateset/21_47_depth.png',cv2.IMREAD_UNCHANGED)
object_depth_colormap = cv2.applyColorMap(cv2.convertScaleAbs(object_depth, alpha=0.2), cv2.COLORMAP_JET)
show_image('',object_depth_colormap)

# 裁剪ROI，即传送带部分
object_depth = object_depth[115:345,0:500]
object_depth_colormap = cv2.applyColorMap(cv2.convertScaleAbs(object_depth, alpha=0.2), cv2.COLORMAP_JET)
show_image('',object_depth_colormap)

In [ ]:
# 将含有物体图像的ROI和原始深度的ROI相减
object_mask_all = cv2.subtract(backgroud_depth,object_depth)
# 提取18cm内的物体
object_mask = cv2.convertScaleAbs(object_mask_all, alpha=1.42)
object_mask_colormap = cv2.applyColorMap(cv2.convertScaleAbs(object_mask, alpha=0.1), cv2.COLORMAP_JET)
show_image('',object_mask_colormap)

In [ ]:
# 二值化提取物体掩码
ret, thresh = cv2.threshold(object_mask,0,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)

# 通过闭运算，滤去传送带上杂波
kernel_1 = cv2.getStructuringElement(cv2.MORPH_RECT,(5,5))
thresh = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel_1,iterations=1)

# 通过开运算，使轮廓区域扩张，进行容错
thresh = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel_1,iterations=1)

# 黑白反转
thresh = cv2.bitwise_not(thresh) 
show_image('',thresh)

In [ ]:
# 读取含有物体的彩色图像
object_RGB = cv2.imread('dateset/21_47_color.png')
object_RGB = object_RGB[115:345,0:500]
show_image('',object_RGB)

### 查找轮廓

In [ ]:
# 查找原始轮廓
_, contours, _ = cv2.findContours(thresh,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE)
object_RGB_1 = object_RGB.copy()
res = cv2.drawContours(object_RGB_1,contours,-1,(0,0,255),2)
show_image('res',res)

In [ ]:
# 将原始轮廓转换成矩形轮廓

rects = []      # 存储矩形框的一个列表

object_RGB_2 = object_RGB.copy()

for contour in contours:
    rects.append(cv2.boundingRect(contour))


    # testing
    x,y,w,h = cv2.boundingRect(contour)
    brcnt = np.array([[[x, y]], [[x+w, y]], [[x+w, y+h]], [[x, y+h]]])
    cv2.drawContours(object_RGB_2, [brcnt], -1, (255,255,255), 2)

show_image('object_RGB',object_RGB_2)

In [ ]:
# 裁剪图像
for i,rect in enumerate(rects):
    x,y,w,h = rect
    img = object_RGB[y-5:y+h+5,x-5:x+w+5]
    show_image('',img)

## 模型训练

使用keras搭建模型

In [ ]:
!pip install keras==2.3.1
%tensorflow_version 1.x
! /opt/bin/nvidia-smi
import keras
import tensorflow as tf
import numpy as np
import cv2
print(keras.__version__)
print(tf.__version__)
print(np.__version__)
print(cv2.__version__)
print(tf.test.is_gpu_available())

     |████████████████████████████████| 378kB 16.9MB/s 
     |████████████████████████████████| 51kB 6.8MB/s 
  Found existing installation: Keras 2.4.3
    Uninstalling Keras-2.4.3:
      Successfully uninstalled Keras-2.4.3
TensorFlow 1.x selected.
Mon Jan 25 06:24:22 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.67       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   64C    P8    11W /  70W |      0MiB / 15079MiB |      0%      Default |
+-------------------------------+----------------------+---------------------

Using TensorFlow backend.


2.3.1
1.15.2
1.19.5
4.1.2
True


In [ ]:
!pip list

In [ ]:
from keras.applications.mobilenet_v2 import MobileNetV2
from keras.models import Sequential
from keras.layers import Conv2D,MaxPool2D,Activation,Dropout,Flatten,Dense
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator,img_to_array,load_img
from keras.callbacks import TensorBoard
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os

In [ ]:
def findAllFile(base):
    for _, _, fs in os.walk(base):
        return fs

In [ ]:
base = '/content/drive/My Drive/Colab Notebooks'
for filename in findAllFile(base):
    print(filename)

test.h5
test.pb
Method_1.ipynb


### 建立MobileNet模型

In [ ]:
# 导入MobileNet网络
mobilenet_model = MobileNetV2(weights='imagenet',include_top=False, input_shape=(128,128,3))

In [ ]:
# 搭建全连接层
top_model = Sequential()
top_model.add(Flatten(input_shape=mobilenet_model.output_shape[1:]))
top_model.add(Dense(256,activation='relu'))
top_model.add(Dropout(0.2))
# !改类别数
top_model.add(Dense(6,activation='softmax'))

In [ ]:
# 拼接模型
model = Sequential()
model.add(mobilenet_model)
model.add(top_model)
model.summary()

### 导入数据

In [ ]:
# 定义数据增强
train_datagen = ImageDataGenerator(
    rotation_range = 180,       # 随机旋转度数
    width_shift_range = 0.05,   # 随机水平平移
    height_shift_range = 0.05,  # 随机竖直平移
    shear_range = 1,            # 随机错切变换
    zoom_range = 0.03,          # 随机放大
    fill_mode = 'nearest',      # 填充方式
    rescale = 1./255,           # 数据归一化
) 
test_datagen = ImageDataGenerator(
    rescale = 1./255,         # 数据归一化
) 

In [ ]:
batch_size = 128

# 生成训练数据
train_generator = train_datagen.flow_from_directory(
    '/content/drive/My Drive/Colab Notebooks/single_img_dataset/train',
    target_size=(128,128),
    batch_size=batch_size,
    class_mode="categorical",
    )

# 测试数据
test_generator = test_datagen.flow_from_directory(
    '/content/drive/My Drive/Colab Notebooks/single_img_dataset/test',
    target_size=(128,128),
    batch_size=batch_size,
    class_mode="categorical"
    )

train_generator.class_indices

### 模型训练

In [ ]:
# 定义优化器，代价函数，训练过程中计算准确率
model.compile(optimizer=SGD(lr=1e-6,momentum=0.95),loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
# 定义tensorboard回调
tbCallBack = TensorBoard(log_dir='./logs',  # log 目录
                 histogram_freq=0,  # 按照何等频率（epoch）来计算直方图，0为不计算
                 batch_size=16,     # 用多大量的数据计算直方图
                 write_graph=True,  # 是否存储网络结构图
                 write_grads=True,  # 是否可视化梯度直方图
                 write_images=True, # 是否可视化参数
                 embeddings_freq=0, 
                 embeddings_layer_names=None, 
                 embeddings_metadata=None
                 )

In [ ]:
# 模型训练
# model = load_model('test.h5')
model.fit_generator(train_generator,
                    steps_per_epoch=len(train_generator),
                    epochs=5,
                    validation_data=test_generator,
                    validation_steps=len(test_generator),
                    # callbacks=[tbCallBack]
                    )

model.save('/content/drive/My Drive/Colab Notebooks/test.h5')

### h5文件转pb文件

In [ ]:
import keras
import tensorflow as tf
import os

In [ ]:
def freeze_session(session, keep_var_names=None, output_names=None, clear_devices=True):
    """
    Freezes the state of a session into a pruned computation graph.
    Creates a new computation graph where variable nodes are replaced by
    constants taking their current value in the session. The new graph will be
    pruned so subgraphs that are not necessary to compute the requested
    outputs are removed.
    @param session The TensorFlow session to be frozen.
    @param keep_var_names A list of variable names that should not be frozen,
                          or None to freeze all the variables in the graph.
    @param output_names Names of the relevant graph outputs.
    @param clear_devices Remove the device directives from the graph for better portability.
    @return The frozen graph definition.
    """
    graph = session.graph
    with graph.as_default():
        freeze_var_names = list(set(v.op.name for v in tf.global_variables()).difference(keep_var_names or []))
        output_names = output_names or []
        output_names += [v.op.name for v in tf.global_variables()]
        input_graph_def = graph.as_graph_def()
        if clear_devices:
            for node in input_graph_def.node:
                node.device = ''
        frozen_graph = tf.graph_util.convert_variables_to_constants(
            session, input_graph_def, output_names, freeze_var_names)
        return frozen_graph

In [ ]:
input_path = '/content/drive/My Drive/Colab Notebooks/'
#keras训练保存的h5文件
input_file = 'test.h5'
weight_file_path = os.path.join(input_path, input_file)
output_graph_name = weight_file_path[:-3] + '.pb'

#  加载模型
keras.backend.set_learning_phase(0)
h5_model = keras.models.load_model(weight_file_path)
frozen_graph = freeze_session(keras.backend.get_session(), output_names=[out.op.name for out in h5_model.outputs])
tf.train.write_graph(frozen_graph, input_path, output_graph_name, as_text=False)
print('Finished')


Instructions for updating:
If using Keras pass *_constraint arguments to layers.

Instructions for updating:
Use `tf.compat.v1.graph_util.convert_variables_to_constants`
Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`
INFO:tensorflow:Froze 430 variables.
INFO:tensorflow:Converted 430 variables to const ops.
Finished


In [ ]:
# !pip install opencv-python==3.4.2.16
# !pip install opencv-contrib-pyton==3.4.2.16
import cv2
print(cv2.__version__)
net = cv2.dnn.readNetFromTensorflow('/content/drive/My Drive/Colab Notebooks/test.pb')

4.1.2


In [ ]:
!pip list

Package                       Version        
----------------------------- ---------------
absl-py                       0.10.0         
alabaster                     0.7.12         
albumentations                0.1.12         
altair                        4.1.0          
argon2-cffi                   20.1.0         
asgiref                       3.3.1          
astor                         0.8.1          
astropy                       4.1            
astunparse                    1.6.3          
async-generator               1.10           
atari-py                      0.2.6          
atomicwrites                  1.4.0          
attrs                         20.3.0         
audioread                     2.1.9          
autograd                      1.3            
Babel                         2.9.0          
backcall                      0.2.0          
backports.tempfile            1.0            
backports.weakref             1.0.post1      
beautifulsoup4                4.6.

## 模型使用

In [ ]:
from keras.models import load_model
import numpy as np
import cv2

In [ ]:
# 加载模型
model = load_model('/content/drive/My Drive/Colab Notebooks/test.h5')
model.summary()

{'BF': 0, 'BS': 1, 'CF': 2, 'CS': 3, 'background': 4, 'obstacle': 5}

In [ ]:
# 读取图片并预测
img = cv2.imread("single_img_dataset/train/CF/0_05_52.png")
img = img/255.0
img = np.resize(img,(1,128,128,3))
result = model.predict_classes(img)[0]
print(result)